In [1]:
import numpy as np
import pickle
from flir_thermal.flirio import flirVideo
import matplotlib.pyplot as plt
import tkinter as tk
import os

In [5]:
mouse = 'KE182'

dates = [#'231231',
        #'240102',
        #'240103',
        #'240104',
        '240105',
        '240106',
        #'240107', >> didn't run
        '240108',
        '240109',
        '240110',
        '240112',
        '240114',
        '240115',
        '240116',
        '240117',
        '240118',
        #'240119', >> no thermal
        '240120',
        '240121',
        '240122',
        #'240123',
        #'240124',
        #'240125',
        #'240126',
        #'240127',
        #'240128',
        #'240129',
        #'240130',
        ]


# Done:

base = str('/mnt/nasquatch/data/2p/kathryn/' + mouse)

fs = 60

# Generate a list of paths 
seq_paths = []
ROI_paths = []
LED_save_paths = []
snout_save_paths = []
background_save_paths = []
plot_save_paths = []

for d in range(len(dates)):
    date_mouse = str(dates[d] + '_' + mouse)
    
    new_path = str(base + '/' + date_mouse + '/')
    seq_paths.append(new_path)
    
    new_ROI = str(new_path + date_mouse + '_expt/thermal_ROI.plk')
    ROI_paths.append(new_ROI)
    
    new_LED = str(new_path + date_mouse + '_expt/LED_temp.plk')
    LED_save_paths.append(new_LED)
    
    new_snout = str(new_path + date_mouse + '_expt/snout_temp.plk')
    snout_save_paths.append(new_snout)
    
    new_background = str(new_path + date_mouse + '_expt/background_temp.plk')
    background_save_paths.append(new_background)
    
    new_plot = str(new_path + date_mouse + '_expt/' + date_mouse + '_plot_of_thermal_ROIs.tiff')
    plot_save_paths.append(new_plot)

In [6]:
snout_save_paths

['/mnt/nasquatch/data/2p/kathryn/KE182/240105_KE182/240105_KE182_expt/snout_temp.plk',
 '/mnt/nasquatch/data/2p/kathryn/KE182/240106_KE182/240106_KE182_expt/snout_temp.plk',
 '/mnt/nasquatch/data/2p/kathryn/KE182/240108_KE182/240108_KE182_expt/snout_temp.plk',
 '/mnt/nasquatch/data/2p/kathryn/KE182/240109_KE182/240109_KE182_expt/snout_temp.plk',
 '/mnt/nasquatch/data/2p/kathryn/KE182/240110_KE182/240110_KE182_expt/snout_temp.plk',
 '/mnt/nasquatch/data/2p/kathryn/KE182/240112_KE182/240112_KE182_expt/snout_temp.plk',
 '/mnt/nasquatch/data/2p/kathryn/KE182/240114_KE182/240114_KE182_expt/snout_temp.plk',
 '/mnt/nasquatch/data/2p/kathryn/KE182/240115_KE182/240115_KE182_expt/snout_temp.plk',
 '/mnt/nasquatch/data/2p/kathryn/KE182/240116_KE182/240116_KE182_expt/snout_temp.plk',
 '/mnt/nasquatch/data/2p/kathryn/KE182/240117_KE182/240117_KE182_expt/snout_temp.plk',
 '/mnt/nasquatch/data/2p/kathryn/KE182/240118_KE182/240118_KE182_expt/snout_temp.plk',
 '/mnt/nasquatch/data/2p/kathryn/KE182/2401

In [8]:
for p in range(len(seq_paths)):
    
    
    # check if we have already extracted temp data for this date:
    if os.path.isfile(snout_save_paths[p]) == True:
        print(str('Already have extracted thermal ROIs for ' + dates[p]))
    else:
        # Load ROIs: 
        with open(ROI_paths[p], 'rb') as f: 
            ROIs = pickle.load(f)

        LED_coords = ROIs[0]
        LED_size = ROIs[1]
        snout_coords = ROIs[2]
        snout_size = ROIs[3]
        background_coords = ROIs[4]
        background_size= ROIs[5]

        # set ROI boundaries
        LED_x = slice(LED_coords[0],(LED_coords[0]+LED_size))
        LED_y = slice(LED_coords[1],(LED_coords[1]+LED_size))
        snout_x = slice(snout_coords[0],(snout_coords[0]+snout_size))
        snout_y = slice(snout_coords[1],(snout_coords[1]+snout_size))
        background_x = slice(background_coords[0],(background_coords[0]+background_size))
        background_y = slice(background_coords[1],(background_coords[1]+background_size))


        all_vids = {}
        # Load all .seq video files in this folder 
        for f in os.listdir(seq_paths[p]): 
            if '.seq' in f: # find all .seq files
                this_vid_path = str(seq_paths[p] + '/' + f) # File to load 
                # this takes forever, but not a ton of memory:
                this_vid = flirVideo(this_vid_path) # Load video info 

        LED_temp  = []
        snout_temp = []
        background_temp = []

        # by frame in this video, extract ROIs, average over space, append to our vars
        for frame in range(1, this_vid.num_frames + 1):
            index, this_frame, times = this_vid.read_video(frames = np.arange(frame, frame+1))
            this_LED_temp = np.nanmean(np.nanmean(this_frame[0, LED_y, LED_x], axis = 0), axis = 0)
            LED_temp.append(this_LED_temp)
            this_s_temp = np.nanmean(np.nanmean(this_frame[0, snout_y, snout_x], axis = 0), axis = 0)
            snout_temp.append(this_s_temp)
            this_b_temp = np.nanmean(np.nanmean(this_frame[0, background_y, background_x], axis = 0), axis = 0)
            background_temp.append(this_b_temp)

        # Plot temp variables + save
        f, ax = plt.subplots(1, 1)
        plt.rcParams['figure.dpi'] = 600
        plt.rcParams['figure.figsize'] = [4, 2]                     
        ds = 10 # plot every ds-th point 
        plt.plot(LED_temp[::ds], c = 'blue')
        plt.plot(snout_temp[::ds], c = 'red')
        plt.plot(background_temp[::ds], c = 'green')
        plt.ylim(15, 40)  
        ax.legend(['IR LED', 'Snout', 'Background'])                
        plt.ylabel('Temp (°C)')
        plt.rcParams['font.size'] = 6                  
        plt.savefig(plot_save_paths[p])

        # Save our temperature variables: 
        with open(LED_save_paths[p], 'wb') as pic:
            pickle.dump(LED_temp, pic)
        pic.close()        
        with open(snout_save_paths[p], 'wb') as pic:
             pickle.dump(snout_temp, pic)
        pic.close()
        with open(background_save_paths[p], 'wb') as pic:
             pickle.dump(background_temp, pic)
        pic.close()       

In [ ]:
    # Sort the video dict alphbetically, to put the runs in order 
    myKeys = list(all_vids.keys())
    myKeys.sort()
    sorted_all_vids = {i: all_vids[i] for i in myKeys}
    del all_vids # free up memory 
    sorted_keys = sorted_all_vids.keys()

    # Concatenate all videos in order 
    concat_vid = np.empty(shape = [0,256,320]) # ineffecient allocation
    for this_key in sorted_keys:
        this_vid = sorted_all_vids[this_key] # pull the video array
        concat_vid = np.append(concat_vid, this_vid, axis = 0) # append it to our concatenated list
        del this_vid
    
    del sorted_all_vids
    
    # Concat vid is frames by x by y 
    # now, extract thermal ROIs and average in space 
    LED_temp = np.mean(np.mean(concat_vid[:, LED_x, LED_y],1),1)
    snout_temp = np.mean(np.mean(concat_vid[:, snout_x, snout_y],1),1)
    background_temp = np.mean(np.mean(imarray[:, background_x, background_y],1),1)
                         
     
